## SQL composition

Logic:
Transform the input to the grammarly correct and executable SQL

- 1. RULE based
- 2. LLM based 

```

kanji<DELETE>, name<KEEP>, builder<GROUP BY>, laid down<DELETE>, launched<DELETE>, completed<DELETE>

```

TODO: where部分如何操作

In [7]:
import random 
import string
import datetime
random.choices(string.ascii_uppercase + string.ascii_lowercase + string.digits, k=20)
datetime.datetime.now().strftime('%m-%d_%H-%M-%S')

'04-02_06-08-10'

In [9]:
from sqlbuilder.query import SELECT, C, T
from sqlbuilder.dummy import dummy_connection, dummy_context
str(SELECT(C.column_name, C.another_column, 123, 'abc').FROM(T.table_name)._as_sql(dummy_connection, dummy_context))
a='test'


<SELECT 'SELECT a', ()>

In [11]:
import re
test_sample = 'round<KEEP> clubs remaining<KEEP> clubs involved<KEEP> winners from previous round<KEEP> new entries this round<DELETE> leagues entering at this round<KEEP>'
matches = re.finditer(r'([^<]*)<([^\s>]*)>', test_sample)
for match in matches:
    print(match.group(0))
    # print(match.group[0])
    print(match.group(1), match.group(2))

round<KEEP>
round KEEP
 clubs remaining<KEEP>
 clubs remaining KEEP
 clubs involved<KEEP>
 clubs involved KEEP
 winners from previous round<KEEP>
 winners from previous round KEEP
 new entries this round<DELETE>
 new entries this round DELETE
 leagues entering at this round<KEEP>
 leagues entering at this round KEEP


In [19]:
import re
import sqlparse
def format_sql(output:str):
    """
    Format subtable output into SQL
    """
    items = output.split(',')
    AGG = ['COUNT', 'AVG', 'SUM', 'MAX', 'MIN', 'KEEP']
    complex = ['GROUP BY', 'ORDER BY']
    select_content = []
    complex_content = []
    for text in items:
        match = re.search(r'(\w+)\s*<([^>]*)>', text)
        item = match.group(1).strip()
        dml = match.group(2).strip()
        if dml in AGG:
            if dml == 'KEEP':
                select_content.append(f'{item}')
            else:
                select_content.append(dml + f'({item})')
        if dml in complex:
            # if dml == 'GROUP BY':
            #     complex_content.append(dml + f' {item}')
            #     select_content.append(f'{item}')
            complex_content.append(dml + f' {item}')
            select_content.append(f'{item}')
    return sqlparse.format('SELECT ' + ' '.join(select_content) + ' '+' '.join(complex_content), keyword_case='upper', reindent=True)

            
    

SELECT name builder
GROUP BY builder


In [4]:
manager.format_sql(test_sample)

'SELECT name builder\nGROUP BY builder'

In [8]:
from sqlalchemy import create_engine 
from sqlalchemy.orm import Session,sessionmaker
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=True)
session = Session(engine)

In [19]:
for i in sqlparse.parse('select * from DF;'):
    print(i.tokens)

[<DML 'select' at 0x7FAB15075240>, <Whitespace ' ' at 0x7FAB15074C40>, <Wildcard '*' at 0x7FAB15074D00>, <Whitespace ' ' at 0x7FAB15074F40>, <Keyword 'from' at 0x7FAB15074EE0>, <Whitespace ' ' at 0x7FAB15074160>, <Identifier 'DF' at 0x7FAB14CDCCF0>, <Punctuation ';' at 0x7FAB150751E0>]


In [2]:
import sqlparse
from typing import List, Any
from sqlalchemy import text
def __sql_parse(sql):
        sql = sql.strip()
        parsed = sqlparse.parse(sql)[0]
        sql_type = parsed.get_type()

        table_name = parsed.get_name()

        first_token = parsed.token_first(skip_ws=True, skip_cm=False)
        ttype = first_token.ttype
        print(
            f"SQL:{sql}, ttype:{ttype}, sql_type:{sql_type}, table:{table_name}"
        )
        return parsed, ttype, sql_type, table_name 
    
def _query(query: str, session, fetch: str = "all"):
        """Run a SQL query and return the results as a list of tuples.

        Args:
            query (str): SQL query to run
            fetch (str): fetch type
        """
        result: List[Any] = []

        print(f"Query[{query}]")
        if not query:
            return result
        cursor = session.execute(text(query))
        if cursor.returns_rows:
            if fetch == "all":
                result = cursor.fetchall()
            elif fetch == "one":
                result = [cursor.fetchone()]
            else:
                raise ValueError("Fetch parameter must be either 'one' or 'all'")
            field_names = tuple(i[0:] for i in cursor.keys())

            result.insert(0, field_names)
            return result
        
def get_simple_fields(self, table_name):
        """Get column fields about specified table."""
        return _query(f"SHOW COLUMNS FROM {table_name}")
    
def run(command: str, session, fetch: str = "all") -> List:
    """Execute a SQL command and return a string representing the results."""
    if not command or len(command) < 0:
        return []
    parsed, ttype, sql_type, table_name = __sql_parse(command)
    if ttype == sqlparse.tokens.DML:
        if sql_type == "SELECT":
            return _query(command, session, fetch)
        else:
            return get_simple_fields(table_name)
        
        


In [2]:
preds = []
import json
with open('./result/data/tabfact_test_04-07_08-05-52.json', 'r') as f:
    lines = f.readlines()
    for l in lines:
        preds.append(json.loads(l)['pred'])
    print(len(preds))

22


## SQL执行

### TEST SQL_Manager

In [1]:
from executor import SQLManager
from sqlalchemy import create_engine
engine = create_engine('sqlite:///db/sqlite/tabfact.db', echo=True)
manager = SQLManager(engine=engine)